# Web Scraping with Python

### Step 1: Install dependencies

In [2]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import os
import requests
from bs4 import BeautifulSoup
import time
import sqlite3

### Step 2: Download HTML

In [4]:
web_page = r'https://en.wikipedia.org/wiki/List_of_Spotify_streaming_records'

In [5]:
response = requests.get(web_page)

In [6]:
response.status_code

200

### Step 3: Transform the HTML

In [7]:
# Read Tables
tables = pd.read_html(web_page)

In [8]:
# Number of tables in the url
print(f'Total tables: {len(tables)}')

Total tables: 27


In [9]:
# Print the first table
data = (tables[0])
print(data)

                  Rank                 Song                      Artist(s)  \
0                    1    "Blinding Lights"                     The Weeknd   
1                    2       "Shape of You"                     Ed Sheeran   
2                    3  "Someone You Loved"                  Lewis Capaldi   
3                    4            "Starboy"       The Weeknd and Daft Punk   
4                    5          "As It Was"                   Harry Styles   
..                 ...                  ...                            ...   
96                  97   "Someone Like You"                          Adele   
97                  98       "Stay with Me"                      Sam Smith   
98                  99             "Havana"  Camila Cabello and Young Thug   
99                 100           "Memories"                       Maroon 5   
100  As of 22 May 2025    As of 22 May 2025              As of 22 May 2025   

    Streams (billions)       Release date               Ref.  


### Step 4: Process the DataFrame

In [10]:
# DataFrame
df = pd.DataFrame(data)
df.head()

,Rank,Song,Artist(s),Streams (billions),Release date,Ref.
0,1,"""Blinding Lights""",The Weeknd,4.848,29 November 2019,[2]
1,2,"""Shape of You""",Ed Sheeran,4.367,6 January 2017,[3]
2,3,"""Someone You Loved""",Lewis Capaldi,3.909,8 November 2018,[4]
3,4,"""Starboy""",The Weeknd and Daft Punk,3.900,21 September 2016,[5]
4,5,"""As It Was""",Harry Styles,3.883,1 April 2022,[6]


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Rank                101 non-null    object
 1   Song                101 non-null    object
 2   Artist(s)           101 non-null    object
 3   Streams (billions)  101 non-null    object
 4   Release date        101 non-null    object
 5   Ref.                101 non-null    object
dtypes: object(6)
memory usage: 4.9+ KB


In [12]:
data_c=data.copy()

In [13]:
data_c = data_c.drop(columns=['Ref.'])
data_c.drop([100], axis=0, inplace=True)

In [14]:
data_c['Release date'] = pd.to_datetime(data_c['Release date'], format='%d %B %Y')
data_c['Streams (billions)'] = data_c['Streams (billions)'].astype(float)
data_c['Rank'] = data_c['Rank'].astype(int)
data_c.head()

,Rank,Song,Artist(s),Streams (billions),Release date
0,1,"""Blinding Lights""",The Weeknd,4.848,2019-11-29
1,2,"""Shape of You""",Ed Sheeran,4.367,2017-01-06
2,3,"""Someone You Loved""",Lewis Capaldi,3.909,2018-11-08
3,4,"""Starboy""",The Weeknd and Daft Punk,3.900,2016-09-21
4,5,"""As It Was""",Harry Styles,3.883,2022-04-01


In [15]:
data_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Rank                100 non-null    int64         
 1   Song                100 non-null    object        
 2   Artist(s)           100 non-null    object        
 3   Streams (billions)  100 non-null    float64       
 4   Release date        100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 4.0+ KB


### Step 5: Store the data in SQLite

In [16]:
database_name = 'spotify.db'

In [17]:
conn = sqlite3.connect(database_name)

In [18]:
table_name = 'most_streamed_songs'

In [20]:
data_c.to_sql(table_name, conn, if_exists='replace', index=False);

In [21]:
cursor = conn.cursor()

In [22]:
pulled_data = cursor.execute('SELECT * FROM most_streamed_songs')

In [23]:
conn.commit()

### Step 6: Visualize the data (optional, but highly recommended)

In [24]:
for row in pulled_data:
    print(f"Rank = {row[0]}, Song = {row[1]}, Artist(s) = {row[2]},  Streams (billions) = {row[3]}, Release date = {row[4]}")

Rank = 1, Song = "Blinding Lights", Artist(s) = The Weeknd,  Streams (billions) = 4.848, Release date = 2019-11-29 00:00:00
Rank = 2, Song = "Shape of You", Artist(s) = Ed Sheeran,  Streams (billions) = 4.367, Release date = 2017-01-06 00:00:00
Rank = 3, Song = "Someone You Loved", Artist(s) = Lewis Capaldi,  Streams (billions) = 3.909, Release date = 2018-11-08 00:00:00
Rank = 4, Song = "Starboy", Artist(s) = The Weeknd and Daft Punk,  Streams (billions) = 3.9, Release date = 2016-09-21 00:00:00
Rank = 5, Song = "As It Was", Artist(s) = Harry Styles,  Streams (billions) = 3.883, Release date = 2022-04-01 00:00:00
Rank = 6, Song = "Sunflower", Artist(s) = Post Malone and Swae Lee,  Streams (billions) = 3.82, Release date = 2018-10-18 00:00:00
Rank = 7, Song = "Sweater Weather", Artist(s) = The Neighbourhood,  Streams (billions) = 3.783, Release date = 2012-12-03 00:00:00
Rank = 8, Song = "One Dance", Artist(s) = Drake with Wizkid and Kyla,  Streams (billions) = 3.638, Release date = 20

In [25]:
conn.close()